# Semantic Search with Amazon OpenSearch 

In [9]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

hub = {
  'HF_MODEL_ID':'distilbert-base-uncased', 
  'HF_TASK':'feature-extraction'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, 
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", 
   py_version="py36", 
)

In [2]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

In [22]:
import numpy
import torch
from transformers import DistilBertTokenizer

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def sentence_to_vector(raw_inputs):
    model_name = "distilbert-base-uncased"
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    inputs_tokens = tokenizer(raw_inputs, padding=True, return_tensors="pt")
    
    data = {
        "inputs": raw_inputs
    }

    # request to get vector data
    response=predictor.predict(data)
    response_tensor = torch.tensor(response)
    
    vector_result = mean_pooling(response_tensor, inputs_tokens['attention_mask'])
    return vector_result


In [29]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3

host = 'search-ultrawarm-test-c7fjh5cny5zgtyxtxm7mwvsequ.us-east-2.es.amazonaws.com' 
region = 'us-east-2' 

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)
index_name = 'nlp_pqa'

es_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [31]:
def semantic_search(raw_inputs):
    search_vector = sentence_to_vector(raw_inputs)[0].tolist()
    query={
        "size": 3,
        "query": {
            "knn": {
                "question_vector":{
                    "vector":search_vector,
                    "k":3
                }
            }
        }
    }

    res = es_client.search(index="nlp_pqa", 
                       body=query,
                       stored_fields=["question","answer"])
    for hit in res['hits']['hits']:
        print("score:",hit['_score'])
        print("question:", hit['fields']['question'])
        print("answer:",hit['fields']['answer'])
        print("\n----------------------------------\n")
        
        

In [35]:
raw_inputs = "does this work with xbox?"
search_vector = sentence_to_vector(raw_inputs)[0].tolist()
#print(search_vector)
semantic_search(raw_inputs)

score: 1.0
question: ['Does this work with xbox?']
answer: ["Hello, this headphone doesn't work with Xbox (there's no USB port), but it can work on PC, laptop, PS4."]

----------------------------------

score: 0.3703665
question: ['Does it work with Xbox?']
answer: ["Yes it should come with a adapter for xbox! Great headset very durable I've had mine for months and still going strong."]

----------------------------------

score: 0.3703665
question: ['Does it work with Xbox?']
answer: ["No, this set is made for PC's and/or phones, mp3 players, etc. The Xbox One doesn't have a 3.5mm jack."]

----------------------------------



In [36]:
raw_inputs = "Can this work with psp4"
semantic_search(raw_inputs)

score: 0.1405858
question: ['Can this be used with psp4']
answer: ['It can if you buy the adapter https://smile.amazon.com/gp/product/B00CPEZ4J6/ref oh aui detailpage o02 s00?ie UTF8&psc 1']

----------------------------------

score: 0.110253416
question: ['Does this work with ps4']
answer: ['Yes it does']

----------------------------------

score: 0.110253416
question: ['Does this work with PS4']
answer: ['yes']

----------------------------------



In [ ]:
# delete endpoint
#predictor.delete_endpoint()